In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.axis as axis

from mpl_toolkits.mplot3d import axes3d
import seaborn as sns

from sklearn import preprocessing
import sklearn.linear_model as skl_lm
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import statsmodels.formula.api as smf


sns.set(font_scale=1.25)
plt.style.use('seaborn-white')

<ipython-input-5-05971dcf0dd4>:19: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/dataframe1.csv'
df = pd.read_csv(file_path)
df.columns
df.drop("Date", axis = 1)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Index(['Date', 'PNL', 'STD_A', 'STD_B', 'STD_C', 'STD_D', 'STD_E', 'STD_F',
       'STD_G', 'STD_Sum', 'STD_Total', 'STD_Diff', 'Open_Vol',
       'Perc_Change_Vol', 'Perc_Change_Price', 'Volume', 'Open_Int', 'Range',
       'High_Diff', 'High_Vol', 'High_PC_Price', 'High_PC_Vol'],
      dtype='object')

divide the data into training and test data sets 

In [ ]:
train_df = df[:258]  # First 258 rows as the training set (all of 2022)
test_df = df[258:]   # Remaining rows as the test set (2023 until May 1st)





In [ ]:
c = len(df.axes[1])

df2 = train_df.copy()


column_names = df2.columns.values.tolist()

for col in column_names[1:]:
  corr_list= []
  corr = train_df[col].corr(train_df['PNL'])
  if corr < 0.2:
    train_df = train_df.drop(col,axis=1)

print(train_df.shape)
train_df

(258, 8)


,Date,PNL,STD_E,STD_F,STD_G,Perc_Change_Price,Volume,Range
0,2023-05-01,-87826.0,34893.760498,1074.230010,124562.410578,1.46,443900.0,2.16
1,2023-04-28,21268.0,34194.618568,21482.950192,121529.229587,2.70,704355.0,2.99
2,2023-04-27,262982.0,25922.699592,13779.058313,118436.921858,0.62,784946.0,1.25
3,2023-04-26,-175431.0,34678.939436,9706.071673,118254.557267,3.59,1084446.0,3.88
4,2023-04-25,44683.0,27595.620454,5983.076914,118929.949403,2.15,784520.0,2.57
...,...,...,...,...,...,...,...,...
253,2022-04-26,-167594.0,13663.182845,17900.130835,110034.694004,1.33,685031.0,2.53
254,2022-04-25,170622.0,15900.755996,13461.507748,113509.334397,2.29,647830.0,3.19
255,2022-04-22,-79995.0,16575.373709,1576.639481,112510.731802,2.42,440249.0,2.95
256,2022-04-21,-138847.0,11167.993145,11568.394751,96018.980693,1.65,551943.0,2.35


In [ ]:
df3  = train_df.drop('PNL', axis =1 )
df3 = df3.drop("Date", axis = 1)
column_names2 = df3.columns.values.tolist()
num_of_col = len(column_names2)
drop_list = []
for i in range(num_of_col):
  first = column_names2[i]
  j= i +1 

  for j in range(j,num_of_col):
  
    second =  column_names2[j]
    corr = df3[first].corr(df3[second])
    if corr > 0.75:
      drop_list.append(second)

drop_list = set(drop_list)
drop_list = list(drop_list)

for i in drop_list:

  train_df = train_df.drop(i,axis=1)

train_df

,Date,PNL,STD_E,STD_F,STD_G,Perc_Change_Price,Volume,Range
0,2023-05-01,-87826.0,34893.760498,1074.230010,124562.410578,1.46,443900.0,2.16
1,2023-04-28,21268.0,34194.618568,21482.950192,121529.229587,2.70,704355.0,2.99
2,2023-04-27,262982.0,25922.699592,13779.058313,118436.921858,0.62,784946.0,1.25
3,2023-04-26,-175431.0,34678.939436,9706.071673,118254.557267,3.59,1084446.0,3.88
4,2023-04-25,44683.0,27595.620454,5983.076914,118929.949403,2.15,784520.0,2.57
...,...,...,...,...,...,...,...,...
253,2022-04-26,-167594.0,13663.182845,17900.130835,110034.694004,1.33,685031.0,2.53
254,2022-04-25,170622.0,15900.755996,13461.507748,113509.334397,2.29,647830.0,3.19
255,2022-04-22,-79995.0,16575.373709,1576.639481,112510.731802,2.42,440249.0,2.95
256,2022-04-21,-138847.0,11167.993145,11568.394751,96018.980693,1.65,551943.0,2.35


no colinearity exists 

In [ ]:
train_df.describe()
y_train_df = train_df.PNL
X_train_df = train_df.drop('PNL', axis=1)

y_test_df = test_df.PNL
X_test_df = test_df.drop('PNL', axis=1)

scaler = preprocessing.MinMaxScaler()
X_train_df = pd.DataFrame(scaler.fit_transform(X_train_df), columns=X_train_df.columns)



scaler = preprocessing.MinMaxScaler()
X_test_df = pd.DataFrame(scaler.fit_transform(X_test_df), columns=X_test_df.columns)



,PNL,STD_E,STD_F,STD_G,Perc_Change_Price,Volume,Range
count,258.000000,258.000000,258.000000,258.000000,258.000000,2.580000e+02,258.000000
mean,-1406.178295,21985.156040,10657.883255,167391.628875,1.739380,7.542844e+05,2.843605
std,246989.833721,7597.634159,9136.075379,84530.805722,1.303185,1.830976e+05,1.086025
min,-777349.000000,4945.843560,875.540100,66382.835194,0.000000,3.969590e+05,0.880000
25%,-172679.500000,16633.143280,4318.372154,95967.674650,0.802500,6.495575e+05,2.152500
50%,-7784.000000,22106.215198,7735.365577,124408.910429,1.465000,7.398335e+05,2.705000
75%,144092.000000,26273.064144,14249.869839,242073.430285,2.417500,8.439288e+05,3.327500
max,851128.000000,39292.703565,65505.993487,393959.791481,8.830000,1.752725e+06,11.210000


In [ ]:
est = smf.ols('PNL ~ STD_E + STD_G + STD_F + Volume + Perc_Change_Price + Range', train_df).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    PNL   R-squared:                       0.245
Model:                            OLS   Adj. R-squared:                  0.227
Method:                 Least Squares   F-statistic:                     13.57
Date:                Tue, 16 May 2023   Prob (F-statistic):           2.49e-13
Time:                        12:49:37   Log-Likelihood:                -3533.0
No. Observations:                 258   AIC:                             7080.
Df Residuals:                     251   BIC:                             7105.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept         -5.417e+05   7.07e+04     -7.662      0.000   -6.81e+05   -4.02e+05
STD_E                 2.9921      2.114      1.416      0.158      -1.171       7.155
STD_G                 0.7990      0.180      4.427      0.000       0.444       1.154
STD_F                 4.5282      1.551      2.920      0.004       1.474       7.582
Volume                0.1853      0.094      1.981      0.049       0.001       0.370
Perc_Change_Price  3731.0104    1.5e+04      0.249      0.804   -2.58e+04    3.33e+04
Range              5.141e+04   2.07e+04      2.489      0.013    1.07e+04    9.21e+04
==============================================================================
Omnibus:                       15.085   Durbin-Watson:                   1.927
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               20.708
Skew:                          -0.422   Prob(JB):                     3.19e-05
Kurtosis:                       4.101   Cond. No.                     4.18e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.18e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:

pred = est.predict(test_df)
test_df['Predict'] = pred
dff = test_df[['PNL', 'Predict']]

RMSE = (((dff.PNL - dff.Predict) ** 2).mean()) ** 0.5


<ipython-input-327-af03fde5e917>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Predict'] = pred


In [ ]:
RMSE

332167.2269035373

RMSE of 320,511 is massive and this is supported by the fact that the R-squared value of the regression is only 0.15; thus the model does not fit the data well 

Next I will verify by Subset evaluation

In [ ]:
import seaborn as sns

from sklearn.preprocessing import scale
import sklearn.linear_model as skl_lm
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import statsmodels.formula.api as smf
import itertools

sns.set(font_scale=1.25)

In [ ]:
y= train_df.PNL
X = train_df.drop(['PNL','Date'], axis=1)
n2 = X.shape[1]
n2
X

6

,STD_E,STD_F,STD_G,Perc_Change_Price,Volume,Range
0,34893.760498,1074.230010,124562.410578,1.46,443900.0,2.16
1,34194.618568,21482.950192,121529.229587,2.70,704355.0,2.99
2,25922.699592,13779.058313,118436.921858,0.62,784946.0,1.25
3,34678.939436,9706.071673,118254.557267,3.59,1084446.0,3.88
4,27595.620454,5983.076914,118929.949403,2.15,784520.0,2.57
...,...,...,...,...,...,...
253,13663.182845,17900.130835,110034.694004,1.33,685031.0,2.53
254,15900.755996,13461.507748,113509.334397,2.29,647830.0,3.19
255,16575.373709,1576.639481,112510.731802,2.42,440249.0,2.95
256,11167.993145,11568.394751,96018.980693,1.65,551943.0,2.35


In [ ]:
def processSubset(feature_set):
    # Fit model on feature_set and calculate RSS
    model = sm.OLS(y,X[list(feature_set)])
    regr = model.fit()
    RMSE = (((regr.predict(X[list(feature_set)]) - y) ** 2).mean()) ** 0.5
    return {"model":regr, "RMSE":RMSE }

In [ ]:
from statsmodels import regression
def getBest(k):
    results = []
    for combo in itertools.combinations(X.columns, k):
        results.append(processSubset(combo))
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    # Choose the model with the lowest RMSE
    best_model = models.loc[models['RMSE'].argmin()]
    print("Processed ", models.shape[0], "models on", k, "predictors.")
    # Return the best model
    return best_model

In [ ]:
models = pd.DataFrame(columns=["RMSE", "model"])
for i in range(1,n2+1):
    models.loc[i] = getBest(i)

Processed  6 models on 1 predictors.
Processed  15 models on 2 predictors.
Processed  20 models on 3 predictors.
Processed  15 models on 4 predictors.
Processed  6 models on 5 predictors.
Processed  1 models on 6 predictors.


In [ ]:
models

,RMSE,model
1,244104.818878,<statsmodels.regression.linear_model.Regressio...
2,242251.239221,<statsmodels.regression.linear_model.Regressio...
3,240358.592089,<statsmodels.regression.linear_model.Regressio...
4,239120.468998,<statsmodels.regression.linear_model.Regressio...
5,237982.637587,<statsmodels.regression.linear_model.Regressio...
6,237949.383555,<statsmodels.regression.linear_model.Regressio...


In [ ]:
models.apply(lambda row: row[1].rsquared, axis=1)

1    0.019456
2    0.034291
3    0.049322
4    0.059091
5    0.068024
6    0.068284
dtype: float64

In [ ]:
best_model = models.loc[6,"model"]
best_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    PNL   R-squared (uncentered):                   0.068
Model:                            OLS   Adj. R-squared (uncentered):              0.046
Method:                 Least Squares   F-statistic:                              3.078
Date:                Tue, 16 May 2023   Prob (F-statistic):                     0.00632
Time:                        12:49:38   Log-Likelihood:                         -3560.1
No. Observations:                 258   AIC:                                      7132.
Df Residuals:                     252   BIC:                                      7153.
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
STD_E                -3.3878      2.154     -1.573      0.117      -7.629       0.854
STD_F                 3.5484      1.713      2.071      0.039       0.175       6.922
STD_G                 0.5743      0.197      2.909      0.004       0.185       0.963
Perc_Change_Price  3.032e+04   1.62e+04      1.875      0.062   -1534.706    6.22e+04
Volume               -0.1030      0.095     -1.085      0.279      -0.290       0.084
Range             -5668.1313   2.14e+04     -0.265      0.791   -4.77e+04    3.64e+04
==============================================================================
Omnibus:                        8.406   Durbin-Watson:                   1.506
Prob(Omnibus):                  0.015   Jarque-Bera (JB):               15.334
Skew:                          -0.012   Prob(JB):                     0.000468
Kurtosis:                       4.194   Cond. No.                     1.29e+06
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.29e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""